<a href="https://colab.research.google.com/github/halynadanchukda/ab-testing-analysis/blob/main/portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A/B Test Analysis

This project analyzes A/B test results to understand how different website changes affect user behavior and key conversion metrics.  
The dataset includes information about user sessions, events, and test groups collected from multiple countries and devices.


## Section 1. Imports

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from google.colab import files

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount ('/content/drive')

# Connecting to the folder in Google Drive
%cd /content/drive/MyDrive

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('bq-results-20250928-185102-1759085792300.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-03,Qatar,mobile,Asia,Organic Search,2,2,new account,1
1,2020-11-03,Ecuador,mobile,Americas,Direct,2,2,new account,1
2,2020-11-12,New Zealand,mobile,Oceania,Undefined,2,2,new account,1
3,2020-11-12,Bulgaria,mobile,Europe,Paid Search,2,2,new account,1
4,2020-11-15,Bulgaria,desktop,Europe,Social Search,2,2,new account,1


## Section 2. Data Analysis

In [ ]:
# Display general information about the DataFrame: number of rows and columns. data types of each column, number of non-null (non-missing) values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  object
 1   country     800996 non-null  object
 2   device      800996 non-null  object
 3   continent   800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  int64 
 6   test_group  800996 non-null  int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 55.0+ MB


In [ ]:
# Count the number of occurrences for each event type
# This helps to understand which user actions (events) appear most frequently in the dataset

events_counts = df['event_name'].value_counts()
events_counts

,count
event_name,
session,107210
session_start,106242
page_view,101907
user_engagement,94520
first_visit,81621
scroll,73643
view_promotion,61695
view_item,44869
session with orders,25892


In [ ]:
# Check which test groups exist in the dataset (e.g., control vs. experiment)
# This helps confirm that the test groups are labeled correctly

test_groups = df['test_group'].unique()
test_groups

array([2, 1])

In [ ]:
# Check which test numbers are included

test_numbers = df['test'].unique()
test_numbers

array([2, 1, 4, 3])

In [ ]:
# Get all unique event names to understand what types of user actions are tracked

events = df['event_name'].unique()
events

array(['new account', 'session with orders', 'session', 'view_item',
       'first_visit', 'user_engagement', 'scroll', 'page_view',
       'view_promotion', 'session_start', 'begin_checkout', 'add_to_cart',
       'select_promotion', 'select_item', 'add_shipping_info',
       'view_search_results', 'add_payment_info', 'click',
       'view_item_list'], dtype=object)

In [ ]:
# Clean up the event names by replacing spaces with underscores
# This makes column names easier to work with in later code

df['event_name'] = df['event_name'].str.replace(' ', '_')

## Section 3. A/B Testing

In [ ]:
# Define the list of metrics to evaluate.
# Each metric has a 'numerator' (event of interest) and a 'denominator' (base event).

metrics = [
    {'metric_name': 'add_payment_info', 'numerator': 'add_payment_info', 'denominator': 'session'},
    {'metric_name': 'add_shipping_info', 'numerator': 'add_shipping_info', 'denominator': 'session'},
    {'metric_name': 'begin_checkout', 'numerator': 'begin_checkout', 'denominator': 'session'},
    {'metric_name': 'new_account', 'numerator': 'new_account', 'denominator': 'session'},
]

# Initialize an empty list to store results for each test and metric
result = []

# Loop through each test in the dataset
for test_number in test_numbers:

  # Filter the dataset for the current test
  data = df[df['test'] == test_number]

  # Loop through each metric to calculate conversions and run statistical tests
  for m in metrics:
    metric = m['metric_name']
    numerator_event = m['numerator']
    denominator_event = m['denominator']

    # Count events for the test group
    numerator_count_test = data[(data['test_group'] == 2) & (data['event_name'] == m['numerator'])]['value'].sum()
    denominator_count_test = data[(data['test_group'] == 2) & (data['event_name'] == m['denominator'])]['value'].sum()

    # Calculate conversion rate for test group
    conversion_test = (numerator_count_test / denominator_count_test)

    # Count events for the control group
    numerator_count_control = data[(data['test_group'] == 1) & (data['event_name'] == m['numerator'])]['value'].sum()
    denominator_count_control = data[(data['test_group'] == 1) & (data['event_name'] == m['denominator'])]['value'].sum()

    # Calculate conversion rate for the control group
    conversion_control = (numerator_count_control / denominator_count_control)

    # Calculate relative change in conversion (% difference between test and control)
    metric_change = ((conversion_test - conversion_control)/conversion_control * 100)

    # Perform Z-test for proportions to check statistical significance
    z_stat, p_value = proportions_ztest([numerator_count_test, numerator_count_control], [denominator_count_test, denominator_count_control])

    # Determine significance based on p-value threshold (α = 0.05)
    if p_value < 0.05:
      significance = True
    else:
      significance = False

    # Append all results for the current metric and test into a list
    result.append({
        'Test number': test_number,
        'Metric': metric,
        'Numerator event': numerator_event,
        'Denominator event': denominator_event,
        'Numerator count': numerator_count_test,
        'Denominator count': denominator_count_test,
        'Conversion rate': conversion_test,
        'Control numerator count': numerator_count_control,
        'Control denominator count': denominator_count_control,
        'Control conversion rate': conversion_control,
        'Metric change': metric_change,
        'z_stat': z_stat,
        'p_value': p_value,
        'Significance': significance

})

# Convert results into a DataFrame for further analysis or export
result_df = pd.DataFrame(result)
# Display the final results table
result_df


,Test number,Metric,Numerator event,Denominator event,Numerator count,Denominator count,Conversion rate,Control numerator count,Control denominator count,Control conversion rate,Metric change,z_stat,p_value,Significance
0,2,add_payment_info,add_payment_info,session,2409,50244,0.047946,2344,50637,0.046290,3.576911,1.240994,0.214608,False
1,2,add_shipping_info,add_shipping_info,session,3510,50244,0.069859,3480,50637,0.068724,1.650995,0.709557,0.477979,False
2,2,begin_checkout,begin_checkout,session,4313,50244,0.085841,4262,50637,0.084168,1.988164,0.952898,0.340642,False
3,2,new_account,new_account,session,4184,50244,0.083274,4165,50637,0.082252,1.241934,0.588793,0.556000,False
4,1,add_payment_info,add_payment_info,session,2229,45193,0.049322,1988,45362,0.043825,12.542021,3.924884,0.000087,True
5,1,add_shipping_info,add_shipping_info,session,3221,45193,0.071272,3034,45362,0.066884,6.560481,2.603571,0.009226,True
6,1,begin_checkout,begin_checkout,session,4021,45193,0.088974,3784,45362,0.083418,6.660587,2.978783,0.002894,True
7,1,new_account,new_account,session,3681,45193,0.081451,3823,45362,0.084278,-3.354299,-1.542883,0.122859,False
8,4,add_payment_info,add_payment_info,session,3601,105141,0.034249,3731,105079,0.035507,-3.541234,-1.571106,0.116158,False
9,4,add_shipping_info,add_shipping_info,session,4956,105141,0.047137,5128,105079,0.048801,-3.411125,-1.785795,0.074132,False


In [ ]:
# Saving scv file

# Save the DataFrame to a CSV file in the Colab environment
filename = 'ab_test_results.csv'
result_df.to_csv(filename, index=False, sep=';')

# Download the file to your local machine
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Section 4. Conclusions

The A/B testing analysis evaluated four experiments across key user behavior metrics — add_payment_info, add_shipping_info, begin_checkout, and new_account. Among these, Test 1 demonstrated statistically significant improvements in conversion rates for all three checkout-related actions, with increases ranging from +6% to +12% and p-values below 0.01. This indicates that the tested changes positively influenced user progression through the checkout funnel.

In contrast, Test 4 showed a small but statistically significant decrease in new account creation (–3.36%), suggesting that the variation may have created problems during the registration process. Tests 2 and 3 produced no statistically significant differences, implying that their respective changes had no measurable effect on user behavior.

Overall, it is recommended to implement the variation from Test 1 and to investigate why changes in Test 4 weren't successful.

Here's the link to the file with final results: https://drive.google.com/file/d/1NCOWddQNhNS2Clsm_H8CKQzieBKPyzPB/view?usp=sharing


#Section 5. Tableau Visualization

Here's the link for interactive data visualisation in Tableau Public:
https://public.tableau.com/app/profile/halyna.danchuk/viz/ABTesting_17591531966100/Story1
